<font size="10" color="black">Análise exploratória de dados</font>

(Exploratory data analysis - EDA)

Eduardo Chaves Ferreira



## O que será tratado no curso

O curso apresenta uma introdução à análise exploratória de dados (EDA), constituída por: 

- visualização
- análise estatística de dados




## Por que fazer EDA?

1- Ter uma melhor compreensão dos dados

2- Compreendendo, estabelecer padrões

3- Baseado nos padrões, propor hipóteses e confirmá-las (ou não)


Algumas vezes não é necessário criar modelos de mineração, a simples análise exploratória é suficiente para extrair as informações que buscamos. 




# 1- Importação de bibliotecas e funções gerais usadas no caderno

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
import scipy.stats as stat
import seaborn as sns
import os
import pandas
import sklearn

from IPython.display import Image
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler


# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos como np.NaN
pandas.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
%matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

# Verificação do local para carga de dados
path = os.environ['PATH']

if path.startswith('C'):
    IN_KAGGLE = False
else:
    IN_KAGGLE = True
    

# Bibliotecas específicas do livro Introduction to Machine Learning with Python
# https://github.com/amueller/introduction_to_ml_with_python
# pip install mglearn

import mglearn


# Configuração do número de linhas e colunas a serem apresentadas em listagens
pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)


In [ ]:
os.listdir('../input')

In [ ]:
# Função de conversão de dados copiada de https://github.com/shakedzy/dython/blob/master/dython/_private.py
# Autor Shaked Zychlinski

def convert(data, to):
    converted = None
    if to == 'array':
        if isinstance(data, np.ndarray):
            converted = data
        elif isinstance(data, pd.Series):
            converted = data.values
        elif isinstance(data, list):
            converted = np.array(data)
        elif isinstance(data, pd.DataFrame):
            converted = data.as_matrix()
    elif to == 'list':
        if isinstance(data, list):
            converted = data
        elif isinstance(data, pd.Series):
            converted = data.values.tolist()
        elif isinstance(data, np.ndarray):
            converted = data.tolist()
    elif to == 'dataframe':
        if isinstance(data, pd.DataFrame):
            converted = data
        elif isinstance(data, np.ndarray):
            converted = pd.DataFrame(data)
    else:
        raise ValueError("Unknown data conversion: {}".format(to))
    if converted is None:
        raise TypeError('cannot handle data conversion of type: {} to {}'.format(type(data),to))
    else:
        return converted

# 2- Carga de dados


## Dados de exemplo

World happiness report (http://worldhappiness.report/).

Somente variáveis numéricas

In [ ]:
if IN_KAGGLE:
    world_happiness = pd.read_csv("../input/world-happiness/2016.csv")
else:
    world_happiness = pd.read_csv("2016.csv")

# Conjunto completo
world_happiness = world_happiness.loc[:,['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual']]



#world_happiness = shuffle(world_happiness).reset_index(drop=True)

# Conjunto resumido para treinamento de modelos
world_happiness_resumido = world_happiness.loc[:,[ 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity']]

# Cria variáveis para treinamento de modelos

colunas_fonte = [ 
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity'
]

colunas_objetivo = [ 
       'Happiness Score'
]

world_happiness_resumido_X = world_happiness_resumido.loc[:,colunas_fonte] 
world_happiness_resumido_Y = world_happiness_resumido.loc[:,colunas_objetivo]


world_happiness.head(35)

## Carrega dados para exercício


Data set de gorgetas com variáveis categóricas

In [ ]:
if IN_KAGGLE:
    tips = pd.read_csv('../input/snstips/tips.csv')
    if 'Unnamed: 0' in tips.columns:
        tips.drop(['Unnamed: 0'], inplace=True, axis=1)
else:
    tips = sns.load_dataset('tips')

tips.head()


In [ ]:
tips2 = tips.copy();

In [ ]:
tips2.columns = ['Valor_Conta','Gorjeta','Sexo','Fumante','Tempo','Horário','Clientes']
tips2.Valor_Conta = tips2.Valor_Conta*4
tips2.Gorjeta = tips2.Gorjeta*4
tips2.head()

Dados sobre tumores (somente informações numéricas)

http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

cancer_data = cancer['data']
# 1 benigno, 0 maligno
cancer_target = cancer['target']
cancer_target_names  = cancer['target_names']
cancer_feature_names = cancer['feature_names']

In [ ]:
cancer_data_DF = pd.DataFrame(cancer_data,columns=cancer_feature_names) 
cancer_data_DF.head()

In [ ]:
cancer_target_DF = pd.DataFrame(cancer_target,columns=['target']) 
cancer_target_DF.head()

# 3 - Obtendo informações gerais sobre os dados

## Dados exemplo

In [ ]:
# Estatísticas básicas
# Verificar mínimos e máximos para garantir se estão dentro dos limites esperados
# Verificar intervalo de variação da medida
# Verificar possíveis outliers

world_happiness.describe()

In [ ]:
# Verificar tipos de dados

world_happiness.dtypes

In [ ]:
# Verificar nulos

world_happiness.info()

## Verificando dados ausentes

Dados ausentes são representados como np.NaN

São dados nulos ou não preenchidos

A função isnull verifica se os dados existem (True) ou não (False)

Há também a função isna, que é idêntica a isnull

CUIDADO: além de dados nulos há também dados que tendem ao infinito (inf ou -inf), fruto, por exemplo, de divisão por zero. Para que sejam considerados nulos usar pandas.options.mode.use_inf_as_na = True

In [ ]:
# Verificar nulos
pandas.options.mode.use_inf_as_na = True
world_happiness.isnull().sum()

In [ ]:
# Para listar dados nulos

df = pd.DataFrame([[0, np.NaN, 2], 
                   [3, 4, np.NaN], 
                   [6, 7, 8]])


df[df.isnull().any(axis=1)]

## Contagem de valores (categóricos ou numéricos)

In [ ]:
world_happiness.Region.value_counts()

In [ ]:
world_happiness.head()

In [ ]:
world_happiness.groupby(['Region']).agg({'Country' : 'count', 'Happiness Rank' : 'mean', 'Happiness Score' : 'mean'}).sort_values(by=['Happiness Score'], ascending=False)


In [ ]:
# Observar a duplicidade de valores

world_happiness['Happiness Rank'].value_counts().head()

## Estatísticas gerais

Observação sobre medidas de forma: 
Curtose (achatamento, onde 0 caracteriza a normal, maior que 0 representa afunilamento e menor que 0 achatamento) e 
Assimetria (skewness, onde 0 caracteriza simetria, maior que 0 caracteriza maior distribuição à direira e menor que 0 caracteriza maior distribuição à esquerda)

In [ ]:

DescribeResult = stats.describe(world_happiness_resumido_X)
minmax = DescribeResult[1]
mean = DescribeResult[2]
variance = DescribeResult[3]
skewness = DescribeResult[4]
kurtosis = DescribeResult[5]

kurtosis

<br>
<font size="8" color="red">EXERCÍCIO</font>

Faça as análises acima para tips.

Usando value_counts, verifique os valores presentes nas variáveis categóricas

In [ ]:
tips.info()

In [ ]:
tips.describe()

In [ ]:
tips.isnull().sum()

In [ ]:
tips.day.value_counts()

In [ ]:
tips.time.value_counts()

In [ ]:
tips.sex.value_counts()

In [ ]:
tips.smoker.value_counts()

<br>
<font size="8" color="red">EXERCÍCIO</font>

Faça as análises acima para cancer_data.

Use value_counts para verificar os valores de target

In [ ]:
cancer_data_DF.info()

In [ ]:
cancer_data_DF.describe()

In [ ]:
cancer_data_DF.isnull().sum()

In [ ]:
DescribeResult = stats.describe(cancer.data)
minmax = DescribeResult[1]
mean = DescribeResult[2]
variance = DescribeResult[3]
skewness = DescribeResult[4]
kurtosis = DescribeResult[5]

kurtosis

In [ ]:
cancer_target_DF.target.value_counts()

# 4- Visualização de Dados

## Visualização usando Matplotlib ou Seaborn?

Matplotlib é a biblioteca mais usada para plotagem. Rápida e fácil de usar. Produz gráficos simples rapidamente e permite a criação de gráficos sofisticados.

https://matplotlib.org/

https://matplotlib.org/tutorials/introductory/sample_plots.html

Seaborn é uma biblioteca baseada na Matplotlib, com recusrsos visuais mais sofisticados, principalmente para análise estatística.

https://seaborn.pydata.org/

https://seaborn.pydata.org/tutorial.html#tutorial

Usaremos ambas no curso


# 4.1 - Visualização Básica

O mínimo necessário para o desenho de gráficos

https://www.oreilly.com/library/view/python-data-science/9781491912126/ch04.html


In [ ]:
# Dados para plotagem
x = np.linspace(0, 10, 100)
x_ = np.linspace(0, 10, 10)

In [ ]:

import matplotlib.pyplot as plt

##### FORMA 1 -  Cria figura com múltiplas imagens sem trabalhar com eixos
fig = plt.figure(figsize=figsize)

# Seleciona local para plot
plt.subplot(2, 1, 1)

# Linha colorida
plt.plot(x, np.sin(x), color='blue', linestyle='-', label='sin(x)')
# Títulos
plt.title('Seno')
plt.xlabel('x')
plt.ylabel('sen')

# Limites de plotagem
plt.xlim(-1, 11)
plt.ylim(-1.5, 1.5)

# Seleciona local para plot
plt.subplot(2, 1, 2)

# Linha gray scale
plt.plot(x, np.cos(x), color='0.4', linestyle='--', label='sin(x)')
plt.title('Cosseno')
plt.xlabel('x')
plt.ylabel('cos')

# Limites de plotagem
plt.xlim(-1, 11)
plt.ylim(-1.5, 1.5)

fig.subplots_adjust(hspace=0.4, wspace=0.4)


plt.tight_layout()
# plt.axis('tight')

plt.show()



In [ ]:
##### FORMA 2 -  Cria figura com múltiplas imagens trabalhando com eixos
fig, ax = plt.subplots(2, 1, figsize=figsize)

ax[0].plot(x, np.sin(x))
ax[0].set_xlabel('x')
ax[0].set_ylabel('sin')
ax[0].set_title('Sin')

ax[1].plot(x, np.cos(x))
ax[1].set_xlabel('x')
ax[1].set_ylabel('cos')
ax[1].set_title('cos')

plt.tight_layout()
plt.show()



In [ ]:
##### FORMA 3 -  Cria figura simples 
fig = plt.figure(figsize=figsize)
ax = plt.axes()

plt.plot(x, np.sin(x), color='blue', linestyle='-', label='sin(x)', linewidth=1)
plt.errorbar(x_, np.sin(x_), yerr=0.1, fmt='.k')
plt.fill_between(x, 1.1*np.sin(x), 0.9*np.sin(x), color='gray', alpha=0.2)


plt.plot(x, np.cos(x), color='0.4', linestyle='--', label='cos(x)', linewidth=4)
plt.scatter(x_, np.cos(x_), marker='o')

# Limites de plotagem
plt.xlim(-1, 11)
plt.ylim(-1.5, 1.5)

# Títulos
plt.title('Seno Cosseno')
plt.xlabel('x')
plt.ylabel('sen cos')

# Anotações
style = dict(size=15, color='gray')
plt.text(6, np.cos(6)+0.2, "Teste", **style)


# Torna legenda visívil
plt.legend()

# Grid
plt.grid(True) 

# Rotaciona labels
plt.xticks(rotation=60)

plt.tight_layout()
plt.show()



# 4.2 - Visualização para análise de dados

## Plotar variável X índice

Pode trazer informações interessantes de tendência se plotarmos variável X índice, mesmo que não trabalhemos com séries temporais. 

No mínimo verificaremos se os dados foram embaralhados

In [ ]:
plt.subplots(figsize=figsize)
plt.plot(range(len(world_happiness)), 
         world_happiness.loc[:,['Happiness Score']],
         'r.'
         )
plt.xlabel('Índice')
plt.ylabel('Happiness Score')
plt.title('Plot ìndice X valor')
plt.grid(True)
plt.show()
plt.tight_layout()

#### <br>
<font size="8" color="red">EXERCÍCIO: Plotar variável X índice</font>

Para tips e cancer_data plotar a variável objetivo X índice



In [ ]:
plt.subplots(figsize=figsize)
plt.plot(range(len(tips)), 
         tips.tip,
         'r.'
         )
plt.xlabel('Índice')
plt.ylabel('Tips')
plt.title('Plot ìndice X Tips')
plt.grid(True)
plt.show()
plt.tight_layout()

In [ ]:
plt.subplots(figsize=figsize)
plt.plot(range(len(cancer_target_DF)), 
         cancer_target_DF.target,
         'r.'
         )
plt.xlabel('Índice')
plt.ylabel('Target')
plt.title('Plot ìndice X Target')
plt.grid(True)
plt.show()
plt.tight_layout()

## Plotar variável X variável

In [ ]:
plt.subplots(figsize=figsize)

wh_ = world_happiness.loc[:,['Economy (GDP per Capita)','Happiness Score']].values

plt.plot(wh_[:,0], 
         wh_[:,1],
         'r.'
         )

plt.xlabel('Economy (GDP per Capita)')
plt.ylabel('Happiness Score')
plt.title('Economy (GDP per Capita) X Happiness Score')
plt.grid(True)
plt.show()
plt.tight_layout()

In [ ]:
# joinplot é limitado em argumentos, preferencialmente usar scatter plot
# há, porém, a vantagem de apresentar as distribuições das variáveis em análise
wh_ = world_happiness.loc[:,['Economy (GDP per Capita)','Happiness Score']].values
with sns.axes_style("whitegrid"):
    g = sns.jointplot(x=wh_[:,0], y=wh_[:,1], height=figsize[0])
    g.set_axis_labels('Economy (GDP per Capita)','Happiness Score')


# Scatter plot

In [ ]:
fig = plt.figure(figsize=figsize)

wh_ = world_happiness.loc[:,['Economy (GDP per Capita)','Happiness Score']].values

plt.scatter(wh_[:,0], wh_[:,1], c=wh_[:,1], s=wh_[:,0]*100, alpha=0.3,
                   cmap='viridis')
plt.colorbar()
plt.xlabel('Economy (GDP per Capita)')
plt.ylabel('Happiness Score')
plt.title('Economy (GDP per Capita) X Happiness Score')
plt.grid(True) 
plt.show()

#### <br>
<font size="8" color="red">EXERCÍCIO: Plotar variável X variável</font>

Para tips, plotar tip X bill. Fazer joinplot com mesmas variáveis. Fazer scatter com as mesmas variáveis, ajustando tamanho e cor por tip.

Para cancer_data, plotar mean radius X mean compactness, ajustando tamanho e cor por target 

In [ ]:
tips2.head()

In [ ]:
plt.subplots(figsize=figsize)

tips_ = tips2.loc[:,['Valor_Conta','Gorjeta']].values

plt.plot(tips_[:,0], 
         tips_[:,1],
         'k.'
         )

plt.xlabel('Valor_Conta')
plt.ylabel('Gorjeta')
plt.title('Valor_Conta X Gorjeta')
plt.grid(True)
plt.show()
plt.tight_layout()

In [ ]:
np.var(tips2.Valor_Conta, ddof=1)

In [ ]:
np.cov(tips2.Valor_Conta,tips2.Gorjeta, ddof=1)

In [ ]:
np.corrcoef(tips2.Valor_Conta,tips2.Gorjeta)

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(np.reshape(tips2.Valor_Conta.values,(len(tips2.Valor_Conta.values),1)),tips2.Gorjeta.values)

# Make predictions using the testing set
tips_pred = regr.predict(np.reshape(tips2.Valor_Conta.values,(len(tips2.Valor_Conta.values),1)))

# The coefficients
print('Coefficients: \n', regr.coef_, regr.intercept_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(tips2.Gorjeta, tips_pred))
print('Soma dos erros: {}'.format(np.sum(tips2.Gorjeta-tips_pred)))
plt.subplots(figsize=figsize)
plt.scatter(tips2.Valor_Conta,tips2.Gorjeta,  color='black')
plt.plot(tips2.Valor_Conta, tips_pred, color='black', linewidth=3)



plt.xlabel('Valor_Conta')
plt.ylabel('Gorjeta')
plt.title('Valor_Conta X Gorjeta')
plt.grid(True)
plt.show()
plt.tight_layout()

In [ ]:

tips_ = tips.loc[:,['total_bill','tip']].values

g = sns.jointplot(x=tips_[:,0], y=tips_[:,1], height=figsize[0])
g.set_axis_labels('total_bill','tip')


In [ ]:
fig = plt.figure(figsize=figsize)

tips_ = tips.loc[:,['total_bill','tip']].values


plt.scatter(tips_[:,0], tips_[:,1], c=tips_[:,1], s=tips_[:,0]*10, alpha=0.3,
                   cmap='viridis')
plt.colorbar()
plt.xlabel('total_bill')
plt.ylabel('tip')
plt.title('total_bill X tip')
plt.grid(True) 
plt.show()

In [ ]:
fig = plt.figure(figsize=figsize)

tips_ = cancer_data_DF.loc[:,['mean radius','mean compactness']].values


plt.scatter(tips_[:,0], tips_[:,1], c=cancer_target_DF.target, s=(cancer_target_DF.target+1)*50, alpha=0.3,
                   cmap='viridis')
plt.colorbar()
plt.xlabel('total_bill')
plt.ylabel('tip')
plt.title('total_bill X tip')
plt.grid(True) 
plt.show()

## Plotagem de variáveis categóricas

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
tips.sex.value_counts().plot(kind='pie', ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize = figsize)

size = 0.3



day_counts = tips.day.value_counts()


cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(len(day_counts))*5)

ax.pie(
    day_counts, 
    radius=1, 
    colors=outer_colors,
    labels = day_counts.index.values,
       wedgeprops=dict(width=size, edgecolor='w'))

#ax.pie(benford, radius=1-size, colors=outer_colors,
#       wedgeprops=dict(width=size, edgecolor='w'))

ax.set(aspect="equal", title='Refeições por dia da semana')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
tips.time.value_counts().plot(kind='bar', ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
sns.barplot(x="day", y="total_bill", hue="sex", data=tips, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
tips.day.value_counts().plot(kind='area', ax=ax)
contadores = tips.day.value_counts()
_ = plt.xticks(range(len(contadores)), contadores.index.values, rotation='vertical')

#### <br>
<font size="8" color="red">EXERCÍCIO: Plotagem de variáveis categóricas</font>

Para tips plotar, na mesma figura, gráficos de barras horizontais, barras verticais, pizza e área para total de tips por smoker, day, sex e time

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=figsize)



tips_ = tips.groupby(['smoker'])['tip'].sum()
ax_ = ax[0,0]
tips_.plot(kind='bar', ax=ax_)
ax_.set_xlabel('Smoker')
ax_.set_ylabel('Tip')
ax_.set_title('Tip X Smoker')

tips_ = tips.groupby(['sex'])['tip'].sum()
ax_ = ax[0,1]
tips_.plot(kind='barh', ax=ax_)
ax_.set_xlabel('Tip')
ax_.set_ylabel('Sex')
ax_.set_title('Tip X Sex')

tips_ = tips.groupby(['day'])['tip'].sum()
ax_ = ax[1,0]
tips_.plot(kind='pie', ax=ax_)
ax_.set_xlabel('Day')
ax_.set_ylabel('Tip')
ax_.set_title('Tip X Day')

tips_ = tips.groupby(['time'])['tip'].sum()
ax_ = ax[1,1]
tips_.plot(kind='area', ax=ax_)
ax_.set_xlabel('Time')
ax_.set_ylabel('Tip')
ax_.set_title('Tip X Time')
contadores = tips.time.value_counts()
_ = plt.xticks(range(len(contadores)), contadores.index.values, rotation='vertical')

fig.subplots_adjust(hspace=0.6, wspace=0.4)
plt.show()

## Mosaico

In [ ]:
tips.groupby(['day','time'])['tip'].count()

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

fig, ax = plt.subplots(1, 1, figsize=figsize)


_=mosaic(tips.loc[:,['day','time']], index =['day','time'], ax=ax, statistic=False, gap=0.01, title='Distribuição de refeições')


#### <br>
<font size="8" color="red">EXERCÍCIO: Mosaico</font>

Para tips, cruzar day e smoker

In [ ]:
import statsmodels
print(statsmodels.__version__)

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

fig, ax = plt.subplots(1, 1, figsize=figsize)


_=mosaic(tips.loc[:,['day','smoker']], index =['day','smoker'], ax=ax, statistic=False, gap=0.01, title='Distribuição de refeições')


## Histogramas

In [ ]:
# Histograma dos dados

fig, ax = plt.subplots(1, 1, figsize=figsize)

world_happiness_resumido.hist(bins=30, color='steelblue', edgecolor='black', linewidth=1.0,
           xlabelsize=8, ylabelsize=8, grid=False, ax=ax)    
plt.tight_layout()   

## Histograma comparando dados de entrada com objetivo

In [ ]:
fig, axes = plt.subplots(6, 1, figsize=figsize)
ruins = world_happiness_resumido.loc[world_happiness_resumido['Happiness Score']<=5.5,:]
bons = world_happiness_resumido.loc[world_happiness_resumido['Happiness Score']>5.5,:]
ax = axes.ravel()
for i in range(6):
    _, bins = np.histogram(world_happiness_resumido.loc[:,colunas_fonte[i]], bins=50)
    ax[i].hist(ruins.loc[:,colunas_fonte[i]], bins=bins, color=mglearn.cm3(0), alpha=.5)
    ax[i].hist(bons.loc[:,colunas_fonte[i]], bins=bins, color=mglearn.cm3(2), alpha=.5)
    ax[i].set_title(colunas_fonte[i],size = 6)
    ax[i].set_yticks(())

ax[0].legend(["tristes", "felizes"], loc="best")


## Histogramas de treinamento e teste

Quando recebemos bases de treinamento e teste já prontas (ex. competições Kaggle) ou quando montamos nossas bases, é importante verificar se pertencem a mesma distribuição


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    world_happiness_resumido_X, world_happiness_resumido_Y, test_size=0.3, random_state=random_state,shuffle =True)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
plt.hist(y_train.T, bins=20, alpha=0.5, label='train')
plt.hist(y_test.T, bins=20, alpha=0.5, label='test')
f.suptitle('Happiness Score - Train Test')
plt.legend(loc='upper right')
plt.show()
plt.tight_layout()

#### <br>
<font size="8" color="red">EXERCÍCIO: Histogramas</font>

Para tips e cancer_data realizar as seguintes análises:
- Histograma dos dados
- Para cancer_data, plotar histograma das variáveis de entrada separando cada gráfico em dois histogramas (maligno e benigno)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)

_ = tips.hist(bins=30, color='steelblue', edgecolor='black', linewidth=1.0,
           xlabelsize=8, ylabelsize=8, grid=False, ax=ax)    
plt.tight_layout()   

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)

_ = cancer_data_DF.hist(bins=30, color='steelblue', edgecolor='black', linewidth=1.0,
           xlabelsize=8, ylabelsize=8, grid=False, ax=ax)    
plt.tight_layout()   

In [ ]:
fig, axes = plt.subplots(15, 2, figsize=figsize)
malignant = cancer.data[cancer.target == 0]
benign = cancer.data[cancer.target == 1]
ax = axes.ravel()
for i in range(30):
    _, bins = np.histogram(cancer.data[:, i], bins=50)
    ax[i].hist(malignant[:, i], bins=bins, color=mglearn.cm3(0), alpha=.5)
    ax[i].hist(benign[:, i], bins=bins, color=mglearn.cm3(2), alpha=.5)
    ax[i].set_title(cancer.feature_names[i],size = 6)
    ax[i].set_yticks(())
ax[0].set_xlabel("Feature magnitude",size = 6)
ax[0].set_ylabel("Frequency",size = 6)
ax[0].legend(["malignant", "benign"], loc="best")


## BoxPlot

Excelente ferramenta para análise de distribuições e outliers

In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
f.suptitle('Happiness Score')

sns.boxplot(x="Region", y="Happiness Score", data=world_happiness,  ax=ax)
ax.set_xlabel("Region",size = 12,alpha=0.8)
ax.set_ylabel("Happiness Score",size = 12,alpha=0.8)
plt.xticks(rotation=60)
plt.tight_layout()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
f.suptitle('world_happiness')

sns.boxplot(data=world_happiness_resumido,  ax=ax)
ax.set_xlabel("Atributos",size = 12,alpha=0.8)
ax.set_ylabel("Valores",size = 12,alpha=0.8)
plt.xticks(rotation=60)
plt.tight_layout()

#### <br>
<font size="8" color="red">EXERCÍCIO: Boxplot</font>

Para tips e cancer_data realizar as seguintes análises:
- Boxplot dos dados
- No caso de tips, boxplot de tip agrupado por time, sex e smoker


In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
f.suptitle('Tips', fontsize=14)

sns.boxplot(x="time", y="tip", data=tips,  ax=ax)
ax.set_xlabel("time",size = 12,alpha=0.8)
ax.set_ylabel("tip",size = 12,alpha=0.8)
plt.xticks(rotation=60)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
tips.boxplot('tip','sex',figsize=figsize, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
tips.boxplot('tip','smoker',figsize=figsize, ax=ax)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
f.suptitle('Tips', fontsize=14)

sns.boxplot(data=tips,  ax=ax)
ax.set_xlabel("Atributos",size = 12,alpha=0.8)
ax.set_ylabel("Valores",size = 12,alpha=0.8)
plt.xticks(rotation=60)
plt.tight_layout()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=figsize)
f.suptitle('Cancer data', fontsize=14)

sns.boxplot(data=cancer_data_DF,  ax=ax)
ax.set_xlabel("Atributos",size = 12,alpha=0.8)
ax.set_ylabel("Valores",size = 12,alpha=0.8)
plt.xticks(rotation=60)
plt.tight_layout()

## Correlações

In [ ]:
# Correlação entre dados

f, ax = plt.subplots(figsize=figsize)
corr = world_happiness_resumido.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Correlação entre variáveis', fontsize=14)

In [ ]:
f, ax = plt.subplots(figsize=figsize)
_ = pd.plotting.scatter_matrix(world_happiness_resumido, ax=ax)



In [ ]:
# Correlação entre dados

pp = sns.pairplot(world_happiness_resumido, size=1.8, aspect=1.8,
                  plot_kws=dict(edgecolor="k", linewidth=0.5),
                  diag_kind="kde", diag_kws=dict(shade=True))

fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('world_happiness', fontsize=14)

#### <br>
<font size="8" color="red">EXERCÍCIO: Correlações</font>

Para tips e cancer_data realizar as seguintes análises:
- Correlações (heatmap) entre dados numéricos de tips e cancer_data


In [ ]:
# Correlação entre dados

f, ax = plt.subplots(figsize=figsize)
corr = tips.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Correlação entre variáveis', fontsize=14)

In [ ]:
# Correlação entre dados

f, ax = plt.subplots(figsize=figsize)
corr = pd.concat([cancer_data_DF, cancer_target_DF], axis=1).corr()
hm = sns.heatmap(round(corr,1), annot=True, ax=ax, cmap="coolwarm",fmt='.0f',
                 linewidths=.05)
f.subplots_adjust(bottom=0.3)
t= f.suptitle('Correlação entre variáveis', fontsize=14)

## Coordenadas paralelas

In [ ]:
from pandas.plotting import parallel_coordinates
fig, ax = plt.subplots(1, 1, figsize=figsize)

parallel_coordinates(frame=world_happiness, class_column='Region'
                     #, color = ('r','g','b','y','w')
                     , ax = ax, cols=['Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'])
plt.xticks(rotation=40)
plt.subplots_adjust(bottom=0.30)

#### <br>
<font size="8" color="red">EXERCÍCIO: Coordenadas paralelas</font>

Para tips e cancer_data realizar as seguintes análises:
- Para cancer_data gráfico de coordenadas paralelas (class_column = target)


In [ ]:
from pandas.plotting import parallel_coordinates
fig, ax = plt.subplots(1, 1, figsize=figsize)

parallel_coordinates(frame=pd.concat([cancer_data_DF, cancer_target_DF], axis=1), 
                     class_column='target'
                     , ax = ax, )
plt.xticks(rotation=60)
fig.subplots_adjust(bottom=0.3)

## Visão 3D dos dados

In [ ]:
# Visão 3D dos dados

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='3d')

xs = world_happiness_resumido['Economy (GDP per Capita)']
ys = world_happiness_resumido['Family']
zs = world_happiness_resumido['Health (Life Expectancy)']
ax.scatter(xs, ys, zs, s=world_happiness_resumido['Happiness Score']**3, alpha=0.6, edgecolors='w')

ax.set_xlabel('Economy (GDP per Capita)')
ax.set_ylabel('Family')
ax.set_zlabel('Health (Life Expectancy)')
plt.tight_layout()

#### <br>
<font size="8" color="red">EXERCÍCIO: Visão 3D</font>

Para tips e cancer_data realizar as seguintes análises:
- Visão 3D de tips usando total_bill, size e tip (tamanho da bolha proporcional à tip)


In [ ]:
# Visão 3D dos dados

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='3d')

xs = tips['total_bill']
ys = tips['size']
zs = tips['tip']
ax.scatter(xs, ys, zs, s=tips['tip']*10, alpha=0.6, edgecolors='w')

ax.set_xlabel('total_bill')
ax.set_ylabel('size')
ax.set_zlabel('tip')
plt.tight_layout()

## Imagens

In [ ]:
#from sklearn.datasets import fetch_lfw_people
#people = fetch_lfw_people(min_faces_per_person=100, resize=1)
# people = people.images[0]
people = np.load('../input/imagenp/imagem.numpy.npy')

### Lista de colormaps
https://matplotlib.org/users/colormaps.html

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.axes()
ax.imshow(people)
ax.set_title('teste')

In [ ]:
plt.imshow(people, cmap="hot")

In [ ]:
plt.imshow(people, cmap="Greys")

## Contornos 2D

In [ ]:
image_shape = people.shape
print(image_shape)
# linspace(start, stop, num)
x = np.linspace(0, image_shape[1], image_shape[1])
# multiplica por -1 caso contrário imagem fica invertida
y = -1*np.linspace(0, image_shape[0], image_shape[0])

In [ ]:
fig = plt.figure(figsize=(9,12))

plt.contour(x,y,people, 30, cmap='RdGy')
plt.tight_layout()

## Contornos 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d')
# meshgrid cria matriz bidimensional cruzando todos os valores de x com y
# X repete x tantas linhas quanto a dimensão de y
X, Y = np.meshgrid(x, y)
# usa log para atenuar profundidade das curvas
ax.contour3D(X, Y, np.log(people), 100)
# Ângulo de visão superior e rotação da imagem
ax.view_init(80, -60)

plt.tight_layout()

## Wire Frame

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d')

X, Y = np.meshgrid(x, y)

ax.plot_wireframe(X, Y, np.log(people))

ax.view_init(80, -60)

plt.tight_layout()

## Superfície

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(9,9))

ax = plt.axes(projection='3d')

X, Y = np.meshgrid(x, y)

ax.plot_surface(X, Y, np.log(people), rstride=1, cstride=1,
                       cmap='viridis', edgecolor='none')

ax.view_init(80, -60)

plt.tight_layout()

## Barras 3D

In [ ]:
# Copiado de https://matplotlib.org/gallery/mplot3d/3d_bars.html

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


# setup the figure and axes
fig = plt.figure(figsize=figsize)
ax1 = fig.add_subplot(121, projection='3d')


# fake data
_x = np.arange(4)
_y = np.arange(5)
_xx, _yy = np.meshgrid(_x, _y)
x, y = _xx.ravel(), _yy.ravel()

top = x + y
bottom = np.zeros_like(top)
width = depth = 1

ax1.bar3d(x, y, bottom, width, depth, top, shade=True)
ax1.set_title('Shaded')


plt.show()

## Mapas

Necessário instalar biblioteca basemap

conda install basemap
https://www.oreilly.com/library/view/python-data-science/9781491912126/ch04.html

In [ ]:
import mpl_toolkits.basemap
mpl_toolkits.basemap.__version__

In [ ]:

from mpl_toolkits.basemap import Basemap


In [ ]:
fig = plt.figure(figsize=(15, 8))
m = Basemap(projection='lcc', resolution='c',
            lat_0=-17, lon_0=-47.92,
            width=8000000, height=5000000)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
x, y = m(-47.92, -15.78)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, 'Brasilia', fontsize=12)

#### <br>
<font size="8" color="red">EXERCÍCIO: Mapas</font>

Plotar todas as capitais de municípios

In [ ]:
os.listdir('../input')

In [ ]:
municipios = pd.read_csv('../input/municipiosbrasileiros/municipios_brasileiros.csv')
if 'Unnamed: 0' in municipios.columns:
    municipios.drop(['Unnamed: 0'], inplace=True, axis=1)
    
municipios_capital = municipios.loc[municipios.capital==1,:]
municipios_capital.reset_index(inplace=True)
municipios_capital.head()

In [ ]:
fig = plt.figure(figsize=(15, 8))
m = Basemap(projection='lcc', resolution='c',
            lat_0=-17, lon_0=-47.92,
            width=8000000, height=5000000)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')
for index, row in municipios_capital.iterrows():
    x, y = m(row['longitude'], row['latitude'])
    plt.plot(x, y, 'ok', markersize=1)
    plt.text(x, y, row['nome_municipio'], fontsize=10)

## Salvando e recuperando imagens

In [ ]:

##### Salva figura
fig.savefig('teste.png')

# Carrega figura
Image('teste.png')    

## Nuvem de palavras

In [ ]:
# https://www.datacamp.com/community/tutorials/wordcloud-python

from wordcloud import WordCloud

In [ ]:
texto = world_happiness.Region.str.cat(sep=' ')


In [ ]:
from collections import Counter
conta_palavras = Counter(texto.split())
conta_palavras = dict(conta_palavras)
del conta_palavras['and']
conta_palavras

In [ ]:
wordcloud = WordCloud(max_font_size=50, repeat=False, background_color="white").generate_from_frequencies(conta_palavras)

In [ ]:
# https://github.com/amueller/word_cloud

#wordcloud = WordCloud(max_font_size=50, repeat=False, background_color="white").generate(texto)
plt.figure(figsize=figsize)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Radar Chart

In [ ]:
world_happiness_ = world_happiness.loc[:,['Region', 
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual']]

world_happiness_.head()


In [ ]:
# copiado de https://python-graph-gallery.com/390-basic-radar-chart/


from math import pi
 
def radar_chart(df):
    # number of variable
    categories=list(df)[1:]
    N = len(categories)

    # We are going to plot the first line of the data frame.
    # But we need to repeat the first value to close the circular graph:
    values=df.loc[0].drop('Region').values.flatten().tolist()
    values += values[:1]
    values

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='grey', size=8)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.ylim(0,3)

    # Plot data
    ax.plot(angles, values, linewidth=1, linestyle='solid')

    # Fill area
    ax.fill(angles, values, 'b', alpha=0.1)


In [ ]:
plt.figure(figsize=figsize)
radar_chart(world_happiness_)

## Rede

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
 
# Build a dataframe with 4 connections
df = pd.DataFrame({ 'from':['A', 'B', 'C','A'], 'to':['D', 'A', 'E','C']})
df
 
# Build your graph
G=nx.from_pandas_edgelist(df, source='from', target='to')
plt.figure(figsize=figsize)
# Plot it
nx.draw(G, with_labels=True)
plt.show()


# Referências

Livros usados como referência:

Introduction to Machine Learning with Python

Python Data Science Handbook (https://www.oreilly.com/library/view/python-data-science/9781491912126/)

Visualização:

https://python-graph-gallery.com/

http://www.apnorton.com/blog/2016/12/19/Visualizing-Multidimensional-Data-in-Python/

https://towardsdatascience.com/the-art-of-effective-visualization-of-multi-dimensional-data-6c7202990c57

https://www.oreilly.com/library/view/python-data-science/9781491912126/ch04.html

https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html